In [1]:
import pandas as pd
import numpy as np
import toml
from pathlib import Path
from sqlalchemy import create_engine

config = toml.load(Path.cwd() / '../../../../configuration/input_configuration.toml')
summary_config = toml.load(Path.cwd() / '../../../../configuration/summary_configuration.toml')

pd.set_option('display.float_format', '{:,.1%}'.format)

In [2]:
# Relative path between notebooks and goruped output directories
output_path = Path(summary_config['sc_run_path']) / summary_config["output_folder"]
survey_path = Path(summary_config['sc_run_path']) / summary_config["survey_folder"]

In [3]:
# add trip type to trip tables
def prep_trip_df(trip):
    df_trip = trip.copy()
    df_trip.rename(columns={'mode':'Mode'}, inplace=True)
    df_trip['Trip Type'] = 'Non-Work'
    df_trip.loc[df_trip['dpurp'] == 'Work', 'Trip Type'] = 'Work'

    # add all tripa
    df = df_trip.copy()
    df['Trip Type'] = 'All Trips'
    df_trip = pd.concat([df_trip.sort_values('Trip Type'), df], ignore_index=True)

    return df_trip

In [4]:
person = pd.read_csv(output_path / 'agg/dash/person_geog.csv')

# counties
trip_county = prep_trip_df(pd.read_csv(output_path / 'agg/dash/mode_share_county.csv'))
# remove "Outside Region"
trip_county = trip_county[trip_county['hh_county'] != 'Outside Region']
# regional geographies
trip_rg = prep_trip_df(pd.read_csv(output_path / 'agg/dash/mode_share_rg.csv'))
# centers
trip_rgc = pd.read_csv(output_path / 'agg/dash/mode_share_rgc.csv')
# add all RGCs
df = trip_rgc[trip_rgc['hh_rgc'] != 'Not in RGC'].groupby(['dpurp','mode']).sum()[['trexpfac']].reset_index()
df['hh_rgc'] = 'All RGCs'
trip_rgc = pd.concat([df, trip_rgc])
trip_rgc = prep_trip_df(trip_rgc)

In [5]:
# equity geographies
equity_geogs = summary_config['hh_equity_geogs']
trip_equity_geog = pd.DataFrame()
for geog in equity_geogs:
    df = prep_trip_df(pd.read_csv(output_path/ ('agg/dash/mode_share_'+geog+'.csv')))
    df['equity_geog'] = df[geog].map({
                                0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population'})
    
    trip_equity_geog = pd.concat([trip_equity_geog, df[['dpurp', 'Mode', 'equity_geog', 'trexpfac', 'Trip Type']]])


## regional mode share

In [6]:
df1 = trip_county.groupby(['Trip Type','Mode'],as_index=False)['trexpfac'].sum()
df2 = trip_county.groupby(['Trip Type'],as_index=False)['trexpfac'].sum()
df2.rename(columns={'trexpfac':'total_trips'}, inplace=True)

df = df1.merge(df2, on='Trip Type')
df['mode_share'] = df['trexpfac'] / df['total_trips']

df.pivot_table(columns='Mode', index='Trip Type', values='mode_share')

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
Trip Type,,,,,,,,
All Trips,1.5%,21.3%,14.1%,44.5%,1.6%,0.3%,2.2%,14.4%
Non-Work,1.5%,23.2%,15.4%,40.4%,1.8%,0.3%,2.2%,15.3%
Work,1.4%,9.4%,5.8%,71.6%,0.0%,0.6%,2.3%,8.9%


## trip mode share by home location

In [7]:
def calc_mode_share(df_trip, geog):
    
    # num trips by mode
    df1 = df_trip.groupby([geog, 'Trip Type', 'Mode'],as_index=False)['trexpfac'].sum()
    # num trips
    df2 = df_trip.groupby([geog, 'Trip Type'],as_index=False)['trexpfac'].sum()
    df2.rename(columns={'trexpfac':'total_trips'}, inplace=True)

    df = df1.merge(df2, on=[geog, 'Trip Type'])
    # mode share
    df['mode_share'] = df['trexpfac'] / df['total_trips']

    # mode share by trip type
    df_all = df.loc[df['Trip Type']=="All Trips"].pivot_table(columns='Mode', index=geog, values='mode_share')
    df_work = df.loc[df['Trip Type']=="Work"].pivot_table(columns='Mode', index=geog, values='mode_share')
    df_non_work = df.loc[df['Trip Type']=="Non-Work"].pivot_table(columns='Mode', index=geog, values='mode_share')
    # num trips by trip type
    trip_all = df.loc[df['Trip Type']=="All Trips"].pivot_table(columns='Mode', index=geog, values='trexpfac')
    trip_work = df.loc[df['Trip Type']=="Work"].pivot_table(columns='Mode', index=geog, values='trexpfac')
    trip_non_work = df.loc[df['Trip Type']=="Non-Work"].pivot_table(columns='Mode', index=geog, values='trexpfac')

    return df_all, df_work, df_non_work, trip_all, trip_work, trip_non_work


df_all, df_work, df_non_work, trip_all, trip_work, trip_non_work = calc_mode_share(trip_county,'hh_county')

In [8]:
df_all

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_county,,,,,,,,
King,1.7%,20.2%,13.2%,43.4%,1.5%,0.4%,3.3%,16.4%
Kitsap,1.2%,22.2%,14.3%,45.2%,1.5%,0.2%,1.3%,14.2%
Pierce,1.4%,23.1%,15.5%,45.0%,1.9%,0.2%,0.8%,12.1%
Snohomish,1.3%,22.5%,15.2%,46.9%,1.7%,0.2%,0.8%,11.4%


In [9]:
trip_all.style.format("{:0,.0f}")

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_county,,,,,,,,
King,"146,497","1,785,686","1,162,461","3,832,507","131,069","36,237","289,483","1,453,432"
Kitsap,"12,448","224,097","144,008","456,750","14,674","2,198","12,749","143,002"
Pierce,"45,694","759,524","511,688","1,482,246","62,395","7,688","24,878","398,128"
Snohomish,"40,489","708,169","480,408","1,478,426","52,870","6,159","26,658","359,454"


In [10]:
df_non_work

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_county,,,,,,,,
King,1.7%,22.0%,14.4%,39.2%,1.7%,0.3%,3.3%,17.3%
Kitsap,1.3%,23.8%,15.3%,41.7%,1.6%,0.2%,0.9%,15.1%
Pierce,1.4%,24.8%,16.8%,40.9%,2.1%,0.2%,0.7%,13.0%
Snohomish,1.3%,24.3%,16.6%,42.5%,1.9%,0.2%,0.9%,12.4%


In [11]:
trip_non_work.style.format("{:0,.0f}")

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_county,,,,,,,,
King,"125,328","1,673,210","1,092,937","2,979,331","131,041","26,493","252,909","1,310,349"
Kitsap,"11,352","213,288","137,272","372,888","14,667","1,979","8,092","134,823"
Pierce,"41,683","721,301","487,679","1,188,575","62,387","6,713","20,146","377,468"
Snohomish,"36,628","667,681","454,805","1,167,121","52,865","5,152","23,971","339,566"


In [12]:
df_work

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_county,,,,,,,,
King,1.7%,9.0%,5.6%,68.5%,0.0%,0.8%,2.9%,11.5%
Kitsap,0.9%,9.4%,5.8%,72.6%,0.0%,0.2%,4.0%,7.1%
Pierce,1.0%,9.9%,6.2%,76.0%,0.0%,0.3%,1.2%,5.3%
Snohomish,1.0%,10.0%,6.3%,76.9%,0.0%,0.2%,0.7%,4.9%


In [13]:
trip_work.style.format("{:0,.0f}")

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_county,,,,,,,,
King,"21,169","112,476","69,524","853,176",28,"9,744","36,574","143,083"
Kitsap,"1,096","10,809","6,736","83,862",7,219,"4,657","8,179"
Pierce,"4,011","38,223","24,009","293,671",8,975,"4,732","20,660"
Snohomish,"3,861","40,488","25,603","311,305",5,"1,007","2,687","19,888"


In [14]:
df_all, df_work, df_non_work, trip_all, trip_work, trip_non_work  = calc_mode_share(trip_rgc, 'hh_rgc')

df_all

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_rgc,,,,,,,,
All RGCs,3.0%,10.7%,5.2%,29.1%,0.5%,1.5%,6.5%,43.4%
Auburn,2.6%,18.6%,10.2%,38.9%,1.1%,0.5%,3.4%,24.7%
Bellevue,2.6%,11.2%,5.1%,31.6%,0.4%,0.9%,3.5%,44.7%
Bothell Canyon Park,1.4%,20.8%,11.4%,51.1%,1.6%,0.4%,0.6%,12.8%
Bremerton,2.5%,14.8%,7.3%,35.7%,0.4%,0.5%,2.2%,36.6%
Burien,2.0%,17.2%,9.5%,44.4%,0.8%,0.4%,3.0%,22.8%
Everett,2.0%,13.4%,6.8%,38.5%,0.7%,0.4%,2.1%,36.1%
Federal Way,1.9%,21.2%,14.9%,35.8%,1.9%,NaN,1.9%,22.4%
Greater Downtown Kirkland,1.9%,17.4%,8.3%,47.3%,0.7%,0.4%,3.8%,20.1%


In [15]:
trip_all.style.format("{:0,.0f}")

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_rgc,,,,,,,,
All RGCs,"33,689","119,959","57,886","325,226","5,836","16,740","72,947","484,041"
Auburn,224,"1,596",875,"3,343",94,39,296,"2,118"
Bellevue,"1,749","7,498","3,391","21,143",300,581,"2,356","29,921"
Bothell Canyon Park,36,552,302,"1,356",43,11,15,339
Bremerton,325,"1,903",933,"4,592",48,62,285,"4,713"
Burien,297,"2,599","1,435","6,697",115,57,449,"3,434"
Everett,520,"3,541","1,802","10,154",185,105,542,"9,503"
Federal Way,44,504,355,851,44,nan,45,533
Greater Downtown Kirkland,604,"5,571","2,672","15,159",236,123,"1,219","6,444"


In [16]:
df_non_work

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_rgc,,,,,,,,
All RGCs,3.0%,11.9%,5.7%,26.2%,0.6%,1.3%,7.0%,44.4%
Auburn,2.5%,19.9%,10.8%,35.8%,1.2%,0.4%,3.1%,26.3%
Bellevue,2.6%,12.3%,5.5%,28.1%,0.5%,0.7%,3.8%,46.4%
Bothell Canyon Park,1.3%,22.5%,12.5%,46.8%,1.9%,0.4%,0.4%,14.2%
Bremerton,2.5%,15.9%,7.7%,32.2%,0.4%,0.5%,1.5%,39.3%
Burien,1.9%,18.6%,10.1%,39.7%,0.9%,0.4%,3.2%,25.3%
Everett,1.9%,14.4%,7.4%,34.1%,0.8%,0.4%,2.2%,38.7%
Federal Way,1.9%,22.2%,16.0%,32.2%,2.1%,NaN,2.0%,23.7%
Greater Downtown Kirkland,1.9%,18.9%,9.1%,43.1%,0.9%,0.4%,3.8%,22.0%


In [17]:
trip_non_work.style.format("{:0,.0f}")

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_rgc,,,,,,,,
All RGCs,"27,537","109,965","53,152","242,982","5,835","11,590","64,440","411,226"
Auburn,191,"1,521",826,"2,734",94,30,234,"2,007"
Bellevue,"1,433","6,886","3,100","15,788",300,402,"2,152","26,025"
Bothell Canyon Park,29,508,282,"1,059",43,10,10,320
Bremerton,277,"1,779",865,"3,596",48,56,172,"4,390"
Burien,241,"2,399","1,299","5,125",115,47,407,"3,271"
Everett,441,"3,288","1,700","7,784",185,90,507,"8,825"
Federal Way,40,469,338,681,44,nan,42,500
Greater Downtown Kirkland,509,"5,173","2,494","11,831",236,102,"1,053","6,026"


In [18]:
df_work

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_rgc,,,,,,,,
All RGCs,3.2%,5.3%,2.5%,43.4%,0.0%,2.7%,4.5%,38.4%
Auburn,3.5%,7.9%,5.2%,64.2%,NaN,0.9%,6.5%,11.7%
Bellevue,2.9%,5.6%,2.7%,49.3%,NaN,1.6%,1.9%,35.9%
Bothell Canyon Park,1.8%,11.2%,5.1%,75.6%,NaN,0.3%,1.3%,4.8%
Bremerton,2.9%,7.4%,4.1%,59.4%,NaN,0.4%,6.7%,19.2%
Burien,2.6%,9.2%,6.2%,72.1%,NaN,0.5%,1.9%,7.5%
Everett,2.2%,7.2%,2.9%,67.1%,NaN,0.4%,1.0%,19.2%
Federal Way,1.5%,13.4%,6.5%,64.9%,NaN,NaN,1.1%,12.6%
Greater Downtown Kirkland,2.1%,8.6%,3.9%,72.3%,NaN,0.5%,3.6%,9.1%


In [19]:
trip_work.style.format("{:0,.0f}")

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_rgc,,,,,,,,
All RGCs,"6,152","9,994","4,734","82,244",1,"5,150","8,507","72,815"
Auburn,33,75,49,609,nan,9,62,111
Bellevue,316,612,291,"5,355",nan,179,204,"3,896"
Bothell Canyon Park,7,44,20,297,nan,1,5,19
Bremerton,48,124,68,996,nan,6,113,323
Burien,56,200,136,"1,572",nan,10,42,163
Everett,79,253,102,"2,370",nan,15,35,678
Federal Way,4,35,17,170,nan,nan,3,33
Greater Downtown Kirkland,95,398,178,"3,328",nan,21,166,418


In [20]:
df_all, df_work, df_non_work, trip_all, trip_work, trip_non_work  = calc_mode_share(trip_rg, 'hh_rg_proposed')

df_all

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_rg_proposed,,,,,,,,
CitiesTowns,1.2%,22.7%,16.5%,45.0%,2.1%,0.2%,0.6%,11.7%
Core,1.4%,22.2%,14.8%,45.7%,1.7%,0.2%,1.8%,12.1%
HCT,1.3%,22.6%,15.3%,46.2%,1.8%,0.2%,1.4%,11.3%
Metro,2.1%,17.8%,10.7%,40.1%,1.2%,0.6%,4.4%,23.2%
UU,1.2%,24.2%,17.7%,44.9%,2.1%,0.2%,0.5%,9.3%


In [21]:
trip_all.style.format("{:0,.0f}")

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_rg_proposed,,,,,,,,
CitiesTowns,"16,621","322,698","234,188","638,813","29,643","2,318","8,248","166,522"
Core,"52,939","835,736","557,570","1,717,337","64,174","8,925","68,351","455,151"
HCT,"46,631","803,539","543,578","1,641,042","63,755","7,633","48,168","401,283"
Metro,"104,043","883,770","530,265","1,996,118","58,244","30,295","218,782","1,155,483"
UU,"7,749","155,870","114,317","289,579","13,654",986,"2,958","59,798"


In [22]:
df_non_work

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_rg_proposed,,,,,,,,
CitiesTowns,1.2%,24.5%,17.9%,40.6%,2.4%,0.2%,0.5%,12.7%
Core,1.4%,24.1%,16.1%,41.3%,2.0%,0.2%,1.8%,13.1%
HCT,1.4%,24.4%,16.6%,41.8%,2.1%,0.2%,1.3%,12.2%
Metro,2.0%,19.4%,11.7%,36.1%,1.4%,0.5%,4.5%,24.4%
UU,1.3%,26.0%,19.2%,40.6%,2.4%,0.1%,0.4%,10.0%


In [23]:
trip_non_work.style.format("{:0,.0f}")

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_rg_proposed,,,,,,,,
CitiesTowns,"15,475","304,470","222,044","504,572","29,637","1,918","6,684","157,919"
Core,"46,986","786,916","526,273","1,351,367","64,157","7,376","58,711","426,744"
HCT,"42,137","758,526","514,786","1,298,656","63,744","6,355","40,752","378,975"
Metro,"87,028","823,705","496,063","1,533,096","58,238","22,143","191,516","1,034,327"
UU,"7,253","148,165","109,234","231,073","13,654",808,"2,288","56,957"


In [24]:
df_work

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_rg_proposed,,,,,,,,
CitiesTowns,0.6%,10.3%,6.9%,76.1%,0.0%,0.2%,0.9%,4.9%
Core,1.2%,9.9%,6.4%,74.4%,0.0%,0.3%,2.0%,5.8%
HCT,1.0%,10.0%,6.4%,75.8%,0.0%,0.3%,1.6%,4.9%
Metro,2.3%,8.2%,4.7%,63.4%,0.0%,1.1%,3.7%,16.6%
UU,0.7%,10.2%,6.7%,77.5%,NaN,0.2%,0.9%,3.8%


In [25]:
trip_work.style.format("{:0,.0f}")

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_rg_proposed,,,,,,,,
CitiesTowns,"1,146","18,228","12,144","134,241",6,400,"1,564","8,603"
Core,"5,953","48,820","31,297","365,970",17,"1,549","9,640","28,407"
HCT,"4,494","45,013","28,792","342,386",11,"1,278","7,416","22,308"
Metro,"17,015","60,065","34,202","463,022",6,"8,152","27,266","121,156"
UU,496,"7,705","5,083","58,506",nan,178,670,"2,841"


In [26]:
equity_geogs = summary_config['hh_equity_geogs']

trip_equity_geog = pd.DataFrame()

for geog in equity_geogs:
    df = prep_trip_df(pd.read_csv(output_path/ ('agg/dash/mode_share_'+geog+'.csv')))
    df['equity_geog'] = df[geog].map({
                                0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population'})
    
    trip_equity_geog = pd.concat([trip_equity_geog, df[['dpurp', 'Mode', 'equity_geog', 'trexpfac', 'Trip Type']]])

df_all, df_work, df_non_work, trip_all, trip_work, trip_non_work  = calc_mode_share(trip_equity_geog, 'equity_geog')

In [27]:
df_all

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
equity_geog,,,,,,,,
Above Regional Average,1.4%,21.7%,14.4%,44.7%,1.6%,0.3%,2.0%,13.8%
Below Regional Average,1.5%,21.0%,13.8%,44.7%,1.6%,0.3%,2.2%,14.8%
Higher Share of Equity Population,1.5%,21.8%,14.6%,43.2%,1.7%,0.3%,2.3%,14.5%


In [28]:
trip_all.style.format("{:0,.0f}")

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
equity_geog,,,,,,,,
Above Regional Average,"411,723","6,168,420","4,088,406","12,693,069","463,821","87,979","558,478","3,929,292"
Below Regional Average,"834,526","11,503,474","7,556,291","24,461,894","849,065","178,895","1,221,019","8,072,916"
Higher Share of Equity Population,"224,519","3,193,010","2,146,717","6,344,671","253,162","46,818","343,111","2,121,888"


In [29]:
df_non_work

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
equity_geog,,,,,,,,
Above Regional Average,1.5%,23.5%,15.6%,40.6%,1.9%,0.3%,2.0%,14.6%
Below Regional Average,1.5%,22.9%,15.1%,40.6%,1.8%,0.3%,2.2%,15.6%
Higher Share of Equity Population,1.6%,23.5%,15.8%,39.2%,2.0%,0.3%,2.4%,15.3%


In [30]:
trip_non_work.style.format("{:0,.0f}")

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
equity_geog,,,,,,,,
Above Regional Average,"363,642","5,816,013","3,866,470","10,026,590","463,766","68,631","486,032","3,616,492"
Below Regional Average,"727,218","10,821,836","7,137,128","19,196,824","848,884","135,677","1,042,945","7,389,581"
Higher Share of Equity Population,"199,086","3,015,079","2,032,584","5,024,130","253,110","37,714","301,731","1,967,163"


In [31]:
df_work

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
equity_geog,,,,,,,,
Above Regional Average,1.3%,9.5%,6.0%,72.2%,0.0%,0.5%,2.0%,8.5%
Below Regional Average,1.5%,9.2%,5.7%,71.4%,0.0%,0.6%,2.4%,9.3%
Higher Share of Equity Population,1.4%,9.7%,6.2%,71.6%,0.0%,0.5%,2.2%,8.4%


In [32]:
trip_work.style.format("{:0,.0f}")

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
equity_geog,,,,,,,,
Above Regional Average,"48,081","352,407","221,936","2,666,479",55,"19,348","72,446","312,800"
Below Regional Average,"107,308","681,638","419,163","5,265,070",181,"43,218","178,074","683,335"
Higher Share of Equity Population,"25,433","177,931","114,133","1,320,541",52,"9,104","41,380","154,725"


## mode share by trip destination

In [33]:
tour_rgc_dest = pd.read_csv(output_path / 'agg/dash/tour_rgc_dest.csv')
trip_rgc_dest = pd.read_csv(output_path / 'agg/dash/trip_rgc_dest.csv')

# order RGCs

# all individual centers
l = tour_rgc_dest['tour_d_rgc'].sort_values().unique().tolist()
l.remove('Not in RGC')
l.remove(np.nan)

# aggregated centers
geog_order = ["Region","In RGC","Not in RGC"]

# combine
geog_order.extend(l)

In [34]:
def get_rgc_location(df, dest_col):
    # regional data
    df_region = df.copy()
    df_region[dest_col] = "Region"

    # aggregated centers
    df_isrgc = df.copy()
    df_isrgc.loc[df_isrgc[dest_col]!="Not in RGC", dest_col] = "In RGC"


    df_rgc = pd.concat([
        df_region,
        df_isrgc,
    # 2025 notes:
    # 7 records in the data are missing both mode and and RGC values
    # remove these records
        df.loc[~df[dest_col].isna()]
        ])

    df_rgc[dest_col] = pd.Categorical(df_rgc[dest_col], ordered=True,
                    categories=geog_order)
    
    return df_rgc

df_tour = get_rgc_location(tour_rgc_dest, 'tour_d_rgc')
df_trip = get_rgc_location(trip_rgc_dest, 'trip_d_rgc')




In [35]:
def calc_mode_share_by_rgc(df_trip_geog, geog_col, mode_col, n_trip_col):

    df = df_trip_geog.pivot_table(columns=mode_col, index=geog_col, values=n_trip_col, aggfunc='sum', observed=True)

    df = df.apply(lambda x: x/ df_trip_geog.groupby(geog_col, observed=True)[n_trip_col].sum())

    return df

\[TRIP\] Mode Share for All Purposes by Regional Center (Destination Location)

In [36]:
calc_mode_share_by_rgc(df_trip, 'trip_d_rgc', 'mode', 'trexpfac')

mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
trip_d_rgc,,,,,,,,
Region,1.5%,21.3%,14.1%,44.5%,1.6%,0.3%,2.2%,14.4%
In RGC,1.9%,15.9%,8.9%,41.9%,0.5%,0.8%,5.6%,24.5%
Not in RGC,1.4%,22.4%,15.1%,45.0%,1.8%,0.2%,1.5%,12.5%
Auburn,1.2%,22.0%,12.4%,48.5%,0.5%,0.2%,3.2%,12.0%
Bellevue,1.7%,15.1%,8.8%,42.8%,0.4%,0.5%,2.6%,28.2%
Bothell Canyon Park,1.0%,22.3%,14.4%,54.0%,1.1%,0.2%,0.4%,6.7%
Bremerton,1.5%,15.8%,9.0%,54.2%,0.1%,0.3%,1.7%,17.3%
Burien,1.1%,22.7%,12.4%,46.6%,0.8%,0.2%,2.2%,13.9%
Everett,1.3%,19.2%,11.1%,47.2%,0.6%,0.3%,1.6%,18.8%


\[TRIP\] Mode Share to Work by Regional Center (Destination Location)

In [37]:
df_trip_work = df_trip[df_trip['dpurp'] == 'Work'].copy()
calc_mode_share_by_rgc(df_trip_work, 'trip_d_rgc', 'mode', 'trexpfac')

mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
trip_d_rgc,,,,,,,,
Region,1.4%,9.4%,5.8%,71.6%,0.0%,0.6%,2.3%,8.9%
In RGC,1.8%,8.5%,5.4%,61.9%,0.0%,1.0%,4.9%,16.5%
Not in RGC,1.2%,9.8%,6.1%,76.6%,0.0%,0.3%,0.9%,5.0%
Auburn,1.0%,9.9%,6.7%,74.0%,NaN,0.2%,1.2%,7.1%
Bellevue,1.4%,9.2%,6.0%,65.0%,NaN,0.6%,1.7%,16.0%
Bothell Canyon Park,1.0%,9.5%,5.7%,80.0%,NaN,0.2%,0.2%,3.5%
Bremerton,1.1%,9.9%,6.5%,74.5%,NaN,0.3%,1.6%,6.2%
Burien,1.5%,10.9%,6.0%,73.2%,NaN,0.4%,1.1%,6.9%
Everett,1.2%,9.3%,5.6%,72.4%,NaN,0.3%,0.5%,10.7%


\[TRIP\] Mode Share for Non-work Purposes by Regional Center (Destination Location)

In [38]:
df_trip_non_work = df_trip[df_trip['dpurp'] != 'Work'].copy()
calc_mode_share_by_rgc(df_trip_non_work, 'trip_d_rgc', 'mode', 'trexpfac')

mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
trip_d_rgc,,,,,,,,
Region,1.5%,23.2%,15.4%,40.4%,1.8%,0.3%,2.2%,15.3%
In RGC,1.9%,18.8%,10.3%,34.3%,0.6%,0.7%,5.8%,27.6%
Not in RGC,1.5%,23.9%,16.2%,41.3%,2.0%,0.2%,1.6%,13.4%
Auburn,1.2%,24.2%,13.5%,43.8%,0.6%,0.2%,3.6%,12.9%
Bellevue,1.8%,17.9%,10.2%,32.0%,0.6%,0.4%,3.0%,34.1%
Bothell Canyon Park,1.0%,27.4%,17.9%,43.6%,1.5%,0.2%,0.5%,8.0%
Bremerton,2.0%,21.2%,11.3%,35.5%,0.2%,0.3%,1.9%,27.6%
Burien,1.1%,24.0%,13.1%,43.7%,0.9%,0.2%,2.4%,14.6%
Everett,1.3%,21.9%,12.7%,40.3%,0.8%,0.3%,1.9%,21.0%


## mode share by tour destination

\[TOUR\] Mode Share for All Purposes by Regional Center (Destination Location)

In [39]:
calc_mode_share_by_rgc(df_tour, 'tour_d_rgc', 'tmodetp', 'toexpfac')

tmodetp,Bike,HOV2,HOV3+,Park,SOV,School Bus,TNC,Transit,Walk
tour_d_rgc,,,,,,,,,
Region,1.4%,22.1%,19.3%,0.1%,37.9%,2.6%,0.5%,3.2%,13.0%
In RGC,1.5%,18.4%,14.2%,0.5%,39.3%,0.6%,1.0%,8.0%,16.3%
Not in RGC,1.3%,23.2%,20.7%,0.1%,37.5%,3.1%,0.3%,1.9%,12.0%
Auburn,0.7%,25.2%,20.6%,0.1%,41.9%,0.3%,0.3%,2.4%,8.6%
Bellevue,1.3%,17.7%,14.4%,0.2%,42.8%,0.3%,0.7%,3.4%,19.1%
Bothell Canyon Park,0.8%,24.2%,21.7%,0.0%,45.0%,1.9%,0.4%,0.8%,5.0%
Bremerton,1.2%,20.9%,15.7%,0.5%,49.9%,0.1%,0.4%,0.8%,10.4%
Burien,0.8%,23.6%,18.3%,0.0%,40.1%,1.6%,0.3%,3.9%,11.3%
Everett,0.9%,22.8%,17.9%,0.1%,40.7%,1.0%,0.4%,2.4%,13.9%


\[TOUR\] Mode Share to Work by Regional Center (Destination Location)

In [40]:
df_tour_work = df_tour[df_tour['pdpurp'] == 'Work'].copy()
calc_mode_share_by_rgc(df_tour_work, 'tour_d_rgc', 'tmodetp', 'toexpfac')

tmodetp,Bike,HOV2,HOV3+,Park,SOV,TNC,Transit,Walk
tour_d_rgc,,,,,,,,
Region,1.4%,15.3%,12.5%,0.5%,62.2%,1.0%,2.7%,4.3%
In RGC,1.8%,13.9%,11.3%,1.1%,55.1%,1.8%,6.0%,8.8%
Not in RGC,1.2%,16.1%,13.1%,0.2%,66.0%,0.6%,1.0%,1.8%
Auburn,1.0%,16.7%,14.7%,0.2%,62.8%,0.5%,1.6%,2.4%
Bellevue,1.5%,15.3%,13.2%,0.4%,59.3%,1.0%,1.9%,7.4%
Bothell Canyon Park,1.0%,16.0%,12.6%,0.1%,68.7%,0.7%,0.3%,0.7%
Bremerton,0.9%,16.9%,13.6%,0.8%,64.4%,0.5%,1.2%,1.7%
Burien,1.2%,16.6%,13.2%,0.2%,63.8%,0.7%,1.5%,2.8%
Everett,1.2%,16.0%,12.9%,0.2%,63.6%,0.6%,0.5%,4.9%


\[TOUR\] Mode Share for Non-work Purposes by Regional Center (Destination Location)

In [41]:
df_tour_non_work = df_tour[df_tour['pdpurp'] != 'Work'].copy()
calc_mode_share_by_rgc(df_tour_non_work, 'tour_d_rgc', 'tmodetp', 'toexpfac')

tmodetp,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
tour_d_rgc,,,,,,,,
Region,1.3%,24.7%,21.9%,28.6%,3.5%,0.2%,3.4%,16.3%
In RGC,1.3%,21.9%,16.5%,27.2%,1.1%,0.4%,9.5%,22.1%
Not in RGC,1.3%,25.3%,23.0%,28.9%,4.1%,0.2%,2.2%,15.1%
Auburn,0.6%,28.1%,22.6%,34.7%,0.4%,0.2%,2.6%,10.8%
Bellevue,1.0%,20.9%,16.1%,22.0%,0.7%,0.2%,5.2%,33.9%
Bothell Canyon Park,0.8%,29.5%,27.5%,30.0%,3.1%,0.2%,1.2%,7.8%
Bremerton,1.8%,27.2%,19.1%,27.3%,0.3%,0.3%,0.2%,23.9%
Burien,0.7%,25.0%,19.3%,35.5%,2.0%,0.2%,4.3%,13.0%
Everett,0.8%,25.7%,20.0%,30.9%,1.4%,0.3%,3.2%,17.7%
